In [31]:
'''
MAIN PROMPT: Investigating if the age of a person has a higher chance of 
winning an Oscar or not for each category
'''
import pandas as pd

df = pd.read_csv("oscar_nominees.csv", encoding="latin1")
df


,Title,Winner,Award,Year
0,Nomadland,Y,Picture,2020
1,Judas and the Black Messiah,N,Picture,2020
2,Mank,N,Picture,2020
3,Minari,N,Picture,2020
4,Promising Young Woman,N,Picture,2020
...,...,...,...,...
2339,Janet Gaynor,Y,Actress,1928
2340,Louise Dresser,N,Actress,1928
2341,Janet Gaynor,N,Actress,1928
2342,Janet Gaynor,N,Actress,1928


In [32]:
# Drop Picture Category
df.drop(df[df["Award"] == "Picture"].index, inplace=True)


In [34]:
categoryList = set(df["Award"])
categoryList

{'Actor', 'Actress', 'Supporting Actor', 'Supporting Actress'}

In [37]:
# Put in ages of all people
from imdb import Cinemagoer

imdbAPI = Cinemagoer()

for category in categoryList:
    # print(f'{category} : {list(df[df["Award"] == category].index)}')
    lst = list(df[df["Award"] == category].index)

    for index in lst:
        

Supporting Actor : [18, 19, 20, 21, 22, 47, 48, 49, 50, 51, 75, 76, 77, 78, 79, 104, 105, 106, 107, 108, 133, 134, 135, 136, 137, 161, 162, 163, 164, 165, 189, 190, 191, 192, 193, 218, 219, 220, 221, 222, 247, 248, 249, 250, 251, 276, 277, 278, 279, 280, 306, 307, 308, 309, 310, 336, 337, 338, 339, 340, 361, 362, 363, 364, 365, 386, 387, 388, 389, 390, 411, 412, 413, 414, 415, 436, 437, 438, 439, 440, 461, 462, 463, 464, 465, 486, 487, 488, 489, 490, 511, 512, 513, 514, 515, 536, 537, 538, 539, 540, 561, 562, 563, 564, 565, 586, 587, 588, 589, 590, 611, 612, 613, 614, 615, 636, 637, 638, 639, 640, 661, 662, 663, 664, 665, 686, 687, 688, 689, 690, 711, 712, 713, 714, 715, 736, 737, 738, 739, 740, 761, 762, 763, 764, 765, 786, 787, 788, 789, 790, 811, 812, 813, 814, 815, 836, 837, 838, 839, 840, 861, 862, 863, 864, 865, 886, 887, 888, 889, 890, 911, 912, 913, 914, 915, 936, 937, 938, 939, 940, 961, 962, 963, 964, 965, 986, 987, 988, 989, 990, 1011, 1012, 1013, 1014, 1015, 1036, 1037, 103